In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [2]:
conn = psycopg2.connect(
    dbname="vmtrans",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
conn.autocommit = True
cursor = conn.cursor()

In [49]:
def find_nearest_segment(cursor : psycopg2.extensions.cursor, lat, lon, point_srid=4326, segment_srid=7844, limit=1):
    # ST_X(ST_StartPoint(geom)) AS start_lon,
    #     ST_Y(ST_StartPoint(geom)) AS start_lat, 
    # ORDER BY geom <-> ST_Transform(ST_SetSRID(ST_MakePoint(%(lon)s, %(lat)s), %(point_srid)s), %(segment_srid)s)
    # ST_Distance(ST_ClosestPoint(geom, 'POINT(%(lon)s %(lat)s)'), 'POINT(%(lon)s %(lat)s)')
    # ST_Distance(ST_ClosestPoint(geom, ST_Transform(ST_SetSRID(ST_MakePoint(%(lon)s, %(lat)s), %(point_srid)s), %(segment_srid)s)), ST_Transform(ST_SetSRID(ST_MakePoint(%(lon)s, %(lat)s), %(point_srid)s), %(segment_srid)s))
    sql = """
    SELECT 
        ufi, from_ufi, to_ufi, ezi_rdname, dir_code, vecaccess, ht_limit, restrictn, 
        ST_AsText(geom) AS geom, 
        ST_Length(ST_Transform(geom, 3857)) AS length_of_multilinestring,
        ST_X(ST_StartPoint(ST_GeometryN(geom, 1))) AS start_lon,
        ST_Y(ST_StartPoint(ST_GeometryN(geom, 1))) AS start_lat,
        CASE
            WHEN ST_NumGeometries(geom) = 1 AND ST_NumPoints(ST_GeometryN(geom, 1)) > 1 THEN
                ST_X(ST_EndPoint(ST_GeometryN(geom, 1)))
            WHEN ST_NumGeometries(geom) > 1 THEN
                ST_X(ST_EndPoint(ST_GeometryN(geom, ST_NumGeometries(geom))))
            ELSE
                ST_X(ST_StartPoint(ST_GeometryN(geom, 1)))
        END AS end_lon,
        CASE
            WHEN ST_NumGeometries(geom) = 1 AND ST_NumPoints(ST_GeometryN(geom, 1)) > 1 THEN
                ST_Y(ST_EndPoint(ST_GeometryN(geom, 1)))
            WHEN ST_NumGeometries(geom) > 1 THEN
                ST_Y(ST_EndPoint(ST_GeometryN(geom, ST_NumGeometries(geom))))
            ELSE
                ST_Y(ST_StartPoint(ST_GeometryN(geom, 1)))
        END AS end_lat
    FROM 
        tr_road_all
    ORDER BY 
        ST_Distance(ST_ClosestPoint(geom, ST_SetSRID(ST_MakePoint(%(lon)s, %(lat)s), %(segment_srid)s)), ST_SetSRID(ST_MakePoint(%(lon)s, %(lat)s), %(segment_srid)s))
    LIMIT %(limit)s;
    """
    cursor.execute(sql, {"lat": lat, "lon": lon, "point_srid": point_srid, "segment_srid": segment_srid, "limit": limit})
    result = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    return result, column_names, cursor.query
result, column_names, query = find_nearest_segment(cursor, -37.91131680842784, 145.12310223093516)
df = pd.DataFrame(result, columns=column_names)
# Get geom coordinates
df


,ufi,from_ufi,to_ufi,ezi_rdname,dir_code,vecaccess,ht_limit,restrictn,geom,length_of_multilinestring,start_lon,start_lat,end_lon,end_lat
0,57676251.0,15536670.0,2354853.0,GLENBROOK AVENUE,B,1,0.0,None,MULTILINESTRING((145.12310854800012 -37.911575...,364.743985,145.123109,-37.911576,145.123872,-37.909116


In [50]:
query

b'\n    SELECT \n        ufi, from_ufi, to_ufi, ezi_rdname, dir_code, vecaccess, ht_limit, restrictn, \n        ST_AsText(geom) AS geom, \n        ST_Length(ST_Transform(geom, 3857)) AS length_of_multilinestring,\n        ST_X(ST_StartPoint(ST_GeometryN(geom, 1))) AS start_lon,\n        ST_Y(ST_StartPoint(ST_GeometryN(geom, 1))) AS start_lat,\n        CASE\n            WHEN ST_NumGeometries(geom) = 1 AND ST_NumPoints(ST_GeometryN(geom, 1)) > 1 THEN\n                ST_X(ST_EndPoint(ST_GeometryN(geom, 1)))\n            WHEN ST_NumGeometries(geom) > 1 THEN\n                ST_X(ST_EndPoint(ST_GeometryN(geom, ST_NumGeometries(geom))))\n            ELSE\n                ST_X(ST_StartPoint(ST_GeometryN(geom, 1)))\n        END AS end_lon,\n        CASE\n            WHEN ST_NumGeometries(geom) = 1 AND ST_NumPoints(ST_GeometryN(geom, 1)) > 1 THEN\n                ST_Y(ST_EndPoint(ST_GeometryN(geom, 1)))\n            WHEN ST_NumGeometries(geom) > 1 THEN\n                ST_Y(ST_EndPoint(ST_Geom

In [51]:
from collections import defaultdict

def get_polygon_segments(segments):
    # Create a dictionary to store segments by their start and end points
    segment_dict = defaultdict(list)
    
    # Populate the dictionary
    for segment in segments:
        start, end = segment
        segment_dict[start].append(end)
        segment_dict[end].append(start)
    
    # Initialize a set to keep track of processed segments
    processed_segments = set()
    
    # Initialize a list to store the polygons
    polygons = []
    
    # Iterate over segments
    for segment in segments:
        if segment in processed_segments:
            continue
        
        # Start a new polygon
        polygon = [segment[0], segment[1]]
        processed_segments.add(segment)
        
        # Explore connected segments to form the polygon
        while True:
            current_point = polygon[-1]
            next_point_candidates = segment_dict[current_point]
            
            # Find the next point that hasn't been visited
            next_point = None
            for candidate in next_point_candidates:
                if (current_point, candidate) not in processed_segments:
                    next_point = candidate
                    break
            
            if next_point is None:
                break
            
            # Add the segment to the polygon
            polygon.append(next_point)
            processed_segments.add((current_point, next_point))
            
            # Check if the polygon is closed
            if next_point == polygon[0]:
                break
        
        # Add the completed polygon to the list
        polygons.append(polygon)
    
    return polygons

# Example usage
segments = [((0, 0), (1, 0)), ((1, 0), (1, 1)), ((1, 1), (0, 1)), ((0, 1), (0, 0)), ((2, 2), (3, 2)), ((3, 2), (3, 3)), ((3, 3), (2, 3)), ((2, 3), (2, 2))]
polygons = get_polygon_segments(segments)
print(polygons)


[[(0, 0), (1, 0), (0, 0)], [(1, 0), (1, 1), (1, 0)], [(1, 1), (0, 1), (1, 1)], [(0, 1), (0, 0), (0, 1)], [(2, 2), (3, 2), (2, 2)], [(3, 2), (3, 3), (3, 2)], [(3, 3), (2, 3), (3, 3)], [(2, 3), (2, 2), (2, 3)]]


In [34]:
from shapely.wkt import loads
k = loads(df['geom'][0])

In [37]:
k.geoms[0].coords[:]

[(145.12310854800012, -37.911575813999946),
 (145.12347844800001, -37.91120991199995),
 (145.12387210600002, -37.90911573999995)]

In [42]:
df['end_lon'][0]

145.12387210600002